In [ ]:
import pandas as pd
 
train = pd.read_csv("input/train.csv")
test = pd.read_csv("input/test.csv")

FileNotFoundError: ignored

In [ ]:
train.head(50)

NameError: ignored

In [ ]:
col_names = list(train.columns.values)
print(col_names)

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

### Data visulatization

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [ ]:
def bar_chart(feature):
    survived = train[train['Survived']==1][feature].value_counts()
    dead = train[train['Survived']==0][feature].value_counts()
    df = pd.DataFrame([survived, dead])
    df.index = ['Survived', 'Dead']
    df.plot(kind='bar', stacked=True, figsize=(10, 5))
    

In [ ]:
from IPython.core.getipython import get_ipython
def create_new_cell(contents):
    
    shell = get_ipython()

    payload = dict(
        source='set_next_input',
        text=contents,
        replace=False,
    )
    shell.payload_manager.write_payload(payload, single=False)

for col in col_names:
    create_new_cell(f"bar_chart('{col}')")

# Feature Engineering

## Name

In [ ]:
train_test_data = [train, test] # combining train and test dataset

for dataset in train_test_data:
    dataset['Title'] = dataset['Name'].str.extract('([A-Z|a-z]+)\.', expand=False)

In [ ]:
test.head()

In [ ]:
train["Title"].value_counts()

In [ ]:
test_titles_count = test['Title'].value_counts()
test_titles = list(list(test_titles_count.__dict__.values())[1].items)
print(test_titles)

In [ ]:
train_titles_count = train['Title'].value_counts()
train_titles = list(list(train_titles_count.__dict__.values())[1].items)
print(train_titles)

In [ ]:
all_titles = list(set(test_titles+train_titles))
all_titles.sort()
print(all_titles)

In [ ]:
titles_mapping = dict()

index = 0
for title in all_titles:
    titles_mapping[title] = index
    index+=1
print(titles_mapping)

In [ ]:
for dataset in train_test_data:
    dataset["Title"] = dataset["Title"].map(titles_mapping)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
bar_chart("Title")

In [ ]:
train.drop("Name", axis=1, inplace=True)
test.drop("Name", axis=1, inplace=True)

In [ ]:
train.head()

## Age

In [ ]:
test.head()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

import numpy as np

def get_ds_wo_nan_target(dataset, features=[]):
    all_ds_wo_nan_targets = dict()
    
    if len(features) == 0:
        features = list(dataset.columns.values)
    
    for feat in features:
        #get all rows without na except the target
        not_feature_targets = list(dataset.columns.values)
        del not_feature_targets[not_feature_targets.index(feat)]
        
        ds_wo_nan_target = dataset
        for not_feat_target in not_feature_targets:
                ds_wo_nan_target = ds_wo_nan_target[ds_wo_nan_target[not_feat_target].notna()]
        
        all_ds_wo_nan_targets[feat] = ds_wo_nan_target
        
    return all_ds_wo_nan_targets

def fill_na_with_model(dataset, features=[]):
    ds_wo_nan_targets = get_ds_wo_nan_target(dataset, features)
    k_fold = KFold(n_splits=10, shuffle=True, random_state=0)
    scoring = "accuracy"
    
    
    for feat in features:
        scores = []
        classifiers = [SVC(), GaussianNB(), RandomForestClassifier(n_estimators=13), DecisionTreeClassifier(), KNeighborsClassifier(n_neighbors = 13)]
        
        data = ds_wo_nan_targets[feat]
        data.columns = list(dataset.columns.values)
        
        train = data[data[feat].notna()]
        target = train[feat]        
        train = train.drop(feat, axis=1)
#         print("train")
#         print(train.shape)
#         print("target")
#         print(target.shape)
        
        
        index = 0
        for clf in classifiers:
            print(f"clf {index}")
            score = cross_val_score(clf, train, target, cv=k_fold, n_jobs=1, scoring=scoring)
            score = round(np.mean(score)*100, 6)
            scores.append(score)
            index += 1
    
        max_score = max(scores)
        max_score_index = scores.index(max_score)
        
#         print(scores)
#         print(max_score_index)
        max_clf = classifiers[max_score_index]
        max_clf.fit(train, target)
        nan_target = data[data[feat].isnull()].drop(feat, axis=1).copy()
        result = max_clf.predict(nan_target)
        
        df_result = pd.DataFrame({feat: result})
        
        print("Feature {} will be filled by model index {} with accuracy {}".format(str(feat), str(max_score_index),  str(max_score)))
        print(result)
        
        for i, ni in enumerate(nan_target.index[:len(df_result)]):
             dataset[feat].loc[ni] = df_result[feat].iloc[i]
        
    return dataset
    

In [ ]:
train["Age"].fillna(train.groupby("Title")["Age"].transform("median"), inplace=True)

In [ ]:
test_median = test.groupby("Title")["Age"].transform("median")
test["Age"].fillna(test_median, inplace=True)
print(test_median)

In [ ]:
for dataset in train_test_data:
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0,
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 26), 'Age'] = 1,
    dataset.loc[(dataset['Age'] > 26) & (dataset['Age'] <= 36), 'Age'] = 2,
    dataset.loc[(dataset['Age'] > 36) & (dataset['Age'] <= 62), 'Age'] = 3,
    dataset.loc[ dataset['Age'] > 62, 'Age'] = 4

In [ ]:
train.head()

In [ ]:
bar_chart("Age")

In [ ]:
test.info()

In [ ]:
train.info()

In [ ]:
import math
test_null = pd.DataFrame(test["Age"].isnull())


test_null[test_null["Age"]!=False]
test["Age"][88] = 17
print(test.loc[[88]])

## Embarked

In [ ]:
Pclass1 = train[train['Pclass']==1]['Embarked'].value_counts()
Pclass2 = train[train['Pclass']==2]['Embarked'].value_counts()
Pclass3 = train[train['Pclass']==3]['Embarked'].value_counts()
df = pd.DataFrame([Pclass1, Pclass2, Pclass3])
df.index = ['1st class','2nd class', '3rd class']
df.plot(kind='bar',stacked=True, figsize=(10,5))

In [ ]:
for dataset in train_test_data:
    dataset["Embarked"] = dataset["Embarked"].fillna("S")

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
test.head()

In [ ]:
test.info()

In [ ]:
train["Embarked"].value_counts()

In [ ]:
test["Embarked"].value_counts()

In [ ]:
embarked_mapping = {"S": 0, "C": 1, "Q": 2}
for dataset in train_test_data:
    dataset["Embarked"] = dataset["Embarked"].map(embarked_mapping)

## Fare

In [ ]:
train.head()

In [ ]:
# fill missing Fare with median fare for each Pclass
train["Fare"].fillna(train.groupby("Pclass")["Fare"].transform("median"), inplace=True)
test["Fare"].fillna(test.groupby("Pclass")["Fare"].transform("median"), inplace=True)
train.head(50)

In [ ]:
train["Fare"].value_counts()

In [ ]:
def categorize_by_step(dataset, feature, step=5):
    minimum = dataset.min(axis=0)[feature]
    maximum = dataset.max(axis=0)[feature]
    print("Feature:", feature)
    print("Min:", minimum)
    print("Max:", maximum)
    
    
    limit = minimum + step
    
    initial_affected_rows = dataset.loc[ dataset[feature] <= limit, feature]
    if initial_affected_rows.shape[0] > 0:
        dataset.loc[ dataset[feature] <= limit, feature] = 0
        print(f"affected row > 0 ({str(initial_affected_rows)}), fare <= {str(limit)} set to 0")
        index = 1
        
    else:
        index = 0
    
    max_before = initial_affected_rows.max(axis=0)
    
    while limit < maximum:
        affected_rows = dataset.loc[ (dataset[feature] > max_before) & (dataset[feature] <= (limit + step)), feature]
        
        if affected_rows.shape[0] > 0:
            print(f"affected row > 0 ({str(affected_rows)}), {str(max_before)} < fare <= {str(limit+step)} set to {str(index)}")
            dataset.loc[ (dataset[feature] > max_before) & (dataset[feature] <= (limit + step)), feature] = index
            max_before = affected_rows.max(axis=0)
            limit = max_before + step
            index += 1
        else:
            limit += step
        
    end_affected_rows = dataset.loc[ dataset[feature] >= maximum, feature].shape[0]
    if end_affected_rows > 0:
        dataset.loc[ dataset[feature] >= maximum, feature] = index
        print(f"affected row > 0 ({str(end_affected_rows)}), fare >= {str(maximum)} set to {str(index)}")
        
    print()
    

In [ ]:
for dataset in train_test_data:
    categorize_by_step(dataset, "Fare", 5)

In [ ]:
train["Fare"].value_counts()

## Cabin

In [ ]:
for dataset in train_test_data:
    dataset["Cabin"] = dataset["Cabin"].str[:1]

In [ ]:
test["Cabin"].value_counts()

In [ ]:
for i in range(1, 4):
    print("Cabin class:", i)
    print(train[train['Pclass']==i]['Cabin'].value_counts())

In [ ]:
cabin_mapping = dict()

import string
index = 0
for l in string.ascii_uppercase:
    cabin_mapping[l] = index
    index+=1
    if l == "F":
        break 
    
print(cabin_mapping)

In [ ]:
for dataset in train_test_data:
    dataset["Cabin"] = dataset["Cabin"].map(cabin_mapping)

In [ ]:
train.head()

In [ ]:
# fill missing Fare with median fare for each Pclass
train["Cabin"].fillna(train.groupby("Pclass")["Cabin"].transform("median"), inplace=True)
test["Cabin"].fillna(test.groupby("Pclass")["Cabin"].transform("median"), inplace=True)

In [ ]:
train.head()

## Sex

In [ ]:
sex_mapping = {"male": 0, "female": 1}
for dataset in train_test_data:
    dataset['Sex'] = dataset['Sex'].map(sex_mapping)

In [ ]:
train.head()

## Family size

In [ ]:
train["SibSp"].value_counts()

In [ ]:
train["FamilySize"] = train["SibSp"] + train["Parch"]
test["FamilySize"] = test["SibSp"] + test["Parch"] 

In [ ]:
train["FamilySize"].value_counts()

## Before Modelling

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.to_csv("train_cleaned.csv")
test.to_csv("test_cleaned.csv")

In [ ]:
features_drop = ['Ticket', 'SibSp', 'Parch']
train = train.drop(features_drop, axis=1)
test = test.drop(features_drop, axis=1)
train = train.drop(['PassengerId'], axis=1)

In [ ]:
train_data = train.drop('Survived', axis=1)
target = train['Survived']

train_data.shape, target.shape

In [ ]:
train_data.info()

In [ ]:
train_data.head()

# Modelling

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

import numpy as np

### Cross Validation (K-fold)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

### KNN

In [ ]:
clf = KNeighborsClassifier(n_neighbors = 13)
scoring = 'accuracy'
score = cross_val_score(clf, train_data, target, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

In [ ]:
# kNN Score
round(np.mean(score)*100, 2)

### Decision Tree

In [ ]:
clf = DecisionTreeClassifier()
scoring = 'accuracy'
score = cross_val_score(clf, train_data, target, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

In [ ]:
# decision tree Score
round(np.mean(score)*100, 2)

### Random Forest

In [ ]:
clf = RandomForestClassifier(n_estimators=13)
scoring = 'accuracy'
score = cross_val_score(clf, train_data, target, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

In [ ]:
# Random Forest Score
round(np.mean(score)*100, 2)

### Naive Bayes

In [ ]:
clf = GaussianNB()
scoring = 'accuracy'
score = cross_val_score(clf, train_data, target, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

In [ ]:
# Naive Bayes Score
round(np.mean(score)*100, 2)

### SVM

In [ ]:
clf = SVC()
scoring = 'accuracy'
score = cross_val_score(clf, train_data, target, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

In [ ]:
round(np.mean(score)*100,2)

In [ ]:
#Splitting the dataset into  training and validation sets
from sklearn.model_selection import train_test_split
training_set, validation_set = train_test_split(train, test_size = 0.2, random_state = 21)

#classifying the predictors and target variables as X and Y
X_train = training_set.iloc[:,0:-1].values
Y_train = training_set.iloc[:,-1].values
X_val = validation_set.iloc[:,0:-1].values
y_val = validation_set.iloc[:,-1].values

In [ ]:
def accuracy(confusion_matrix):
   diagonal_sum = confusion_matrix.trace()
   sum_of_all_elements = confusion_matrix.sum()
   return diagonal_sum / sum_of_all_elements

from sklearn.neural_network import MLPClassifier
mlp0 = MLPClassifier(hidden_layer_sizes=(8,8,8), activation='relu', solver='adam', max_iter=1000)
mlp1 = MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=300,activation = 'relu',solver='adam',random_state=1)
mlp0.fit(X_train, Y_train)
y_pred = mlp0.predict(X_val)

from sklearn.metrics import confusion_matrix
#Comparing the predictions against the actual observations in y_val
cm = confusion_matrix(y_pred, y_val)

#Printing the accuracy
print("Accuracy of MLPClassifier : ", accuracy(cm))


# Testting

In [ ]:
clf = RandomForestClassifier(n_estimators=13)
clf.fit(train_data, target)

test_data = test.drop("PassengerId", axis=1).copy()
test_data.head()

In [ ]:
prediction = clf.predict(test_data)

In [ ]:
submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": prediction
    })

submission.to_csv('submission_my.csv', index=False)

In [ ]:
submission = pd.read_csv('submission_my.csv')
submission.head()